# Machine Learning

In [1]:
# global imports
import numpy as np
import pandas as pd
# visualization
import altair as alt
from scipy.stats import gaussian_kde
# model selection
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error
# machine learning models
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

# suppress scientific notation
pd.options.display.float_format = '{:.2f}'.format

## Import Data

In [2]:
# import data frames
%store -r scaled_dfs
# create data frames
X_train = scaled_dfs[0]
y_train = scaled_dfs[1]
X_test = scaled_dfs[2]
y_test = scaled_dfs[3]

## Machine Learning Tools

In [3]:
# function for evaluating machine learning model
def evaluate_model(model):
    # fit model
    model.fit(X_train,y_train.values.ravel())
    # obtain predictions
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    # get rmse values
    rmse_train = mean_squared_error(y_train.values.ravel(),train_pred,squared=False)
    rmse_test = mean_squared_error(y_test.values.ravel(),test_pred,squared=False)
    # print rmse
    print('RMSE for train: ',rmse_train)
    print('RMSE for test: ',rmse_test)
    # return predictions
    return test_pred

In [4]:
# function for visualizing y and y hat
def visualize(y_pred):
    x = y_test.values.ravel()
    y = y_pred
    xy = np.vstack([x,y])
    z = gaussian_kde(xy)(xy)
    idx = z.argsort()
    x, y, z = x[idx], y[idx], z[idx]
    graph_dict = {'Observed':x,'Predicted':y,'Color': z}
    graph = pd.DataFrame(data=graph_dict)
    scatter = alt.Chart(graph,title='Scatterplot of Observed and Predicted Values').mark_point().encode(
        x=alt.X('Observed:Q',scale=alt.Scale(zero=False)),
        y=alt.Y('Predicted:Q',scale=alt.Scale(zero=False)),
        color=alt.Color('Color:Q',scale=alt.Scale(scheme='blues'))
    )
    points = np.linspace(min(y_test.values.ravel()),max(y_test.values.ravel()),5)
    points_dict = {'x':points,'y':points}
    points_df = pd.DataFrame(data=points_dict)
    line = alt.Chart(points_df).mark_line(color='red',opacity=0.5).encode(
        x=alt.X('x:Q',title='Observed Life Expectancy',scale=alt.Scale(zero=False)),
        y=alt.Y('y:Q',title='Predicted Life Expectancy',scale=alt.Scale(zero=False)),
    )
    (scatter+line).properties(height=300,width=500).display()

## K Nearest Neighbors

K nearest neighbors predicts $\hat{y}=\frac{1}{K}\sum_{k=1}^Kw_ky_k$ as the weighted average of the life expectancy for the $k$ closest points in the training set.

***Cross Validation***

Tune `n_neighbors` and `weights`.

In [5]:
# knn model
knn = KNeighborsRegressor()
# parameter grid
knn_pars = {'n_neighbors':[2,3,5,7,10,15,20],'weights':['uniform','distance']}
# grid search
knn_grid = GridSearchCV(estimator=knn,
                        param_grid=knn_pars,
                        cv=KFold(random_state=130,shuffle=True),
                        scoring='neg_root_mean_squared_error')\
                       .fit(X_train, y_train.values.ravel())
# output best parameters from grid search
knn_grid.best_params_

{'n_neighbors': 5, 'weights': 'distance'}

***Final Model***

In [6]:
# evaluate best model
knn_model = KNeighborsRegressor(n_neighbors=5,weights='distance')
knn_pred = evaluate_model(knn_model)

RMSE for train:  0.0
RMSE for test:  2.6484076983389673


***Visualize Results***

In [7]:
# create graph
visualize(knn_pred)

alt.LayerChart(...)

## Random Forest

Algorithm for random forest:
1. Obtain $B$ bootstrap samples of the dataset.
2. Grow a decision tree for each bootstrap sample. At each split, randomly sample one-third of the predictors and choose the best cutpoint. 
3. Average the $B$ trees to get the final prediction. 

***Cross Validation***

Tune `n_etsimators`.

In [8]:
# rf model
rf = RandomForestRegressor(random_state=134,max_features=(1/3))
# parameter grid
rf_pars = {'n_estimators':[50,100,300,500,700,1000]}
# grid search
rf_grid = GridSearchCV(estimator=rf,
                       param_grid=rf_pars,
                       cv=KFold(random_state=130,shuffle=True),
                       scoring='neg_root_mean_squared_error')\
                      .fit(X_train, y_train.values.ravel())
# output best parameters from grid search
rf_grid.best_params_

{'n_estimators': 500}

***Final Model***

In [9]:
# evaluate best model
rf_model = RandomForestRegressor(random_state=134,max_features=(1/3),n_estimators=500)
rf_pred = evaluate_model(rf_model)

RMSE for train:  0.7918462143123466
RMSE for test:  2.1873961104921507


***Visuualize Results***

In [10]:
# create graph
visualize(rf_pred)

alt.LayerChart(...)

## Gradient Boosted Trees

Gradient boosted trees build $B$ weak learning trees with a given maximum depth. Weak learning classifiers are built based off of the information from previous trees, meaning $F_b(x) = F_{b-1}(x)+h_b(x)$. $h_b(x)=\underset{x}{\arg\min}\sum_{i=1}^n\ell(y_i,F_{b-1}(x_i)+h(x_i))$ minimizes the loss function. In this model, I used least squares as the loss function. The final prediction is chosen as $\hat{y}=F_{B}(x)=\sum_{b=1}^{B}h_b(x)$.

***Cross Validation***

Tune `n_estimators`, `learning_rate`, and `max_depth`.

In [11]:
# gb model
gb = GradientBoostingRegressor(random_state=205,max_features=(1/3))
# parameter grid
gb_pars = {'n_estimators':[50,100,300,500,700,1000],
           'learning_rate':[0.01,0.1,0.5,0.9],
           'max_depth':[2,3,5,10]}
# grid search
gb_grid = GridSearchCV(estimator=gb,
                        param_grid=gb_pars,
                        cv=KFold(random_state=130,shuffle=True),
                        scoring='neg_root_mean_squared_error')\
                       .fit(X_train, y_train.values.ravel())
# output best parameters from grid search
gb_grid.best_params_

{'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 1000}

***Final Model***

In [12]:
# evaluate best model
gb_model = GradientBoostingRegressor(random_state=205,
                                     max_features=(1/3),
                                     n_estimators=1000,
                                     learning_rate=0.01,
                                     max_depth=10)
gb_pred = evaluate_model(gb_model)

RMSE for train:  0.037753195886366865
RMSE for test:  2.0291997521598617


***Visualize Results***

In [13]:
# create graph
visualize(gb_pred)

alt.LayerChart(...)

## Linear Regression

The equation for linear regression is $\hat{y}=\beta_0+\beta_1X_1+\dots+\beta_pX_p$.

### Ordinary Least Squares

***Final Model***

In [14]:
# fit model
ols_model = LinearRegression()
ols_pred = evaluate_model(ols_model)

RMSE for train:  3.9127863165363226
RMSE for test:  3.8164729886520514


***Visualize Results***

In [15]:
# create graph
visualize(ols_pred)

alt.LayerChart(...)

### Lasso

Lasso finds coefficients that minimize $L\big[\sum_{i=1}^n\big(y_i-(\beta_0+\sum_{j=1}^p\beta_jx_{ij})\big)\big] + \lambda\sum_{i=1}^p|\beta_i|$.

***Cross Validation***

Tune `alpha`.

In [16]:
# lasso model
lasso = Lasso()
# parameter grid
lasso_pars = {'alpha':[0.01,0.05,0.07,0.1,0.5,1]}
# grid search
lasso_grid = GridSearchCV(estimator=lasso,
                          param_grid=lasso_pars,
                          cv=KFold(random_state=130,shuffle=True),
                          scoring='neg_root_mean_squared_error')\
                          .fit(X_train, y_train.values.ravel())
# output best parameters from grid search
lasso_grid.best_params_

{'alpha': 0.07}

***Final Model***

In [17]:
# fit model
lasso_model = Lasso(alpha=0.07)
lasso_pred = evaluate_model(lasso_model)

RMSE for train:  3.9251678345576044
RMSE for test:  3.822889186407458


***Visualize Results***

In [18]:
# create graph
visualize(lasso_pred)

alt.LayerChart(...)

### Ridge

Ridge finds coefficients that minimize $L\big[\sum_{i=1}^n\big(y_i-(\beta_0+\sum_{j=1}^p\beta_jx_{ij})\big)\big] + \lambda\sum_{i=1}^p\beta_i^2$.

***Cross Validation***

Tune `alpha`.

In [19]:
# ridge model
ridge = Ridge()
# parameter grid
ridge_pars = {'alpha':[0.1,1,10,20,30,50,70,100]}
# grid search
ridge_grid = GridSearchCV(estimator=ridge,
                          param_grid=ridge_pars,
                          cv=KFold(random_state=130,shuffle=True),
                          scoring='neg_root_mean_squared_error')\
                          .fit(X_train, y_train.values.ravel())
# output best parameters from grid search
ridge_grid.best_params_

{'alpha': 70}

***Final Model***

In [20]:
# fit model
ridge_model = Ridge(alpha=70)
ridge_pred = evaluate_model(ridge_model)

RMSE for train:  3.915124592030618
RMSE for test:  3.8130626484928647


***Visualize Results***

In [21]:
# create graph
visualize(ridge_pred)

alt.LayerChart(...)

### Elastic Net

Elastic net finds coefficients that minimize $L\big[\sum_{i=1}^n\big(y_i-(\beta_0+\sum_{j=1}^p\beta_jx_{ij})\big)\big] + \lambda_1\sum_{i=1}^p|\beta_i| + \lambda_2\sum_{i=1}^p\beta_i^2$.

***Cross Validation***

Tune `alpha` and `l1_ratio`

In [22]:
# ridge model
en = ElasticNet()
# parameter grid
en_pars = {'alpha':[0.001,0.01,0.1,1,10,20,50],'l1_ratio':[0.25,0.5,0.75]}
# grid search
en_grid = GridSearchCV(estimator=en,
                       param_grid=en_pars,
                       cv=KFold(random_state=130,shuffle=True),
                       scoring='neg_root_mean_squared_error')\
                       .fit(X_train, y_train.values.ravel())
# output best parameters from grid search
en_grid.best_params_

{'alpha': 0.1, 'l1_ratio': 0.75}

***Final Model***

In [23]:
# fit model
en_model = ElasticNet(alpha=0.1,l1_ratio=0.75)
en_pred = evaluate_model(en_model)

RMSE for train:  3.927580252712444
RMSE for test:  3.8230861755355066


***Visualize Results***

In [24]:
# create graph
visualize(en_pred)

alt.LayerChart(...)

## Support Vector Machine

The equation for support vector machine regression is $L(\alpha)=\frac{1}{2}\sum_{i=1}^N\sum_{j=1}^N(\alpha_i-\alpha_i^*)(\alpha_j-\alpha_j^*)K(x_ix_j) + \epsilon\sum_{i=1}^N(\alpha_i+\alpha) - \sum_{i=1}^Ny_i(\alpha_i-\alpha_i^*)$. This equation is subject to the constraints that $\sum_{n=1}^N(\alpha_n-\alpha_n^*)=0$, $\forall n: 0 \leq \alpha_n \leq C$, and $\forall n: 0 \leq \alpha_n^*\leq C$. $\epsilon$ is the slack parameter and $C$ is the budget tuning parameter. 
- Linear kernel: $K(x_i,x_j)=\sum_{n=1}^Nx_ix_j$
- Polynomial kernel: $K(x_i,x_j)=(r+\gamma\sum_{n=1}^Nx_ix_j)^d$
- Radial basis function kernel: $K(x_i,x_j)=exp(-\gamma\sum_{n=1}^N(x_i-x_j)^2)$
- Sigmoid kernel: $K(x_i,x_j)=tanh(r+\gamma\sum_{n=1}^Nx_ix_j)$

***Cross Validation***

Tune `C`, `epsilon`, and `kernel` plus parameters associated with each kernel.
- `degree` for poly
- `gamma` for rbf, poly, and sigmoid
- `coef0` for poly and sigmoid

My original cross validation took several hours to run, so I shortened the code for GitHub. Here is the original parameter map:

`svr_pars = {'C':[0.1,1,10,50,100],
            'epsilon':[0.001,0.01,0.1,1,10],
            'kernel': ['linear','poly','rbf','sigmoid'],
            'degree': [2,3],
            'gamma': ['scale','auto'],
            'coef0': [0,1,10]}`

In [25]:
# svr model
svr = SVR(kernel='rbf')
# parameter grid
svr_pars = {'C':[0.1,1,10,50,100],
            'epsilon':[0.001,0.01,0.1,1,10],
            'gamma': ['scale','auto']}
# grid search
svr_grid = GridSearchCV(estimator=svr,
                        param_grid=svr_pars,
                        cv=KFold(random_state=130,shuffle=True),
                        scoring='neg_root_mean_squared_error')\
                       .fit(X_train, y_train.values.ravel())
# output best parameters from grid search
svr_grid.best_params_

{'C': 100, 'epsilon': 1, 'gamma': 'auto'}

***Final Model***

In [26]:
# evaluate best model
svr_model = SVR(C=100, epsilon=1, kernel='rbf',gamma='auto')
svr_pred = evaluate_model(svr_model)

RMSE for train:  1.8789301041835018
RMSE for test:  2.6506688462512797


***Visualize Results***

In [27]:
# create graph
visualize(svr_pred)

alt.LayerChart(...)

## Store Data

The gradient boosted tree model had the lowest RMSE so I will use this as my final model.

In [28]:
# create final model
final_model = GradientBoostingRegressor(random_state=205,
                                        max_features=(1/3),
                                        n_estimators=1000,
                                        learning_rate=0.01,
                                        max_depth=10)
# fit final model
final_model.fit(X_train,y_train.values.ravel())
# obtain predictions
train_pred = final_model.predict(X_train)

In [29]:
# create list of items related to the model
model_lst = [final_model,train_pred,gb_pred]
# store data
%store model_lst

Stored 'model_lst' (list)
